# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,12.74,53,100,0.89,US,1731275228
1,1,sumberpucung,-8.1586,112.4829,21.83,96,100,0.65,ID,1731275229
2,2,grytviken,-54.2811,-36.5092,-0.47,84,64,2.74,GS,1731275230
3,3,imbituba,-28.2400,-48.6703,19.89,72,0,7.00,BR,1731275231
4,4,tutoia,-2.7619,-42.2744,26.58,81,87,6.57,BR,1731275232


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True, 
    tiles="OSM", 
    size="Humidity", 
    color="City", 
    alpha=0.5, 
    hover_cols=["City", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 11) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,guerrero negro,27.9769,-114.0611,23.80,33,97,5.07,MX,1731275259
53,53,papatowai,-46.5619,169.4708,11.57,49,77,7.24,NZ,1731275289
62,62,faya-largeau,17.9257,19.1043,22.18,28,30,7.68,TD,1731275299
74,74,port pirie,-33.1833,138.0167,24.31,42,71,2.37,AU,1731275314
84,84,aioun,16.6614,-9.6149,26.95,12,2,4.98,MR,1731275326
95,95,broken hill,-31.9500,141.4333,22.37,38,100,3.99,AU,1731275338
97,97,laguna,38.4210,-121.4238,22.26,32,0,0.45,US,1731275341
102,102,isla vista,34.4133,-119.8610,24.64,37,59,1.34,US,1731275346
114,114,karratha,-20.7377,116.8463,25.88,39,40,3.46,AU,1731275360
119,119,masterton,-40.9597,175.6575,17.83,48,73,2.38,NZ,1731275365


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,guerrero negro,MX,27.9769,-114.0611,33,
53,papatowai,NZ,-46.5619,169.4708,49,
62,faya-largeau,TD,17.9257,19.1043,28,
74,port pirie,AU,-33.1833,138.0167,42,
84,aioun,MR,16.6614,-9.6149,12,
95,broken hill,AU,-31.9500,141.4333,38,
97,laguna,US,38.4210,-121.4238,32,
102,isla vista,US,34.4133,-119.8610,37,
114,karratha,AU,-20.7377,116.8463,39,
119,masterton,NZ,-40.9597,175.6575,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
 
limit = 1
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
papatowai - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
port pirie - nearest hotel: Ellen Hotel
aioun - nearest hotel: فندق العيون
broken hill - nearest hotel: Ibis Styles
laguna - nearest hotel: Holiday Inn Express & Suites
isla vista - nearest hotel: The Club & Guest House
karratha - nearest hotel: Karratha International Hotel
masterton - nearest hotel: Copthorne Hotel Solway Park
biltine - nearest hotel: فندق ثلاث نجوم بلتن
nouadhibou - nearest hotel: Noadhibou Guesthouse
sirsa - nearest hotel: Hotel Gumber
taoudenni - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
mhamid - nearest hotel: Hotel Kasbah Azalay
la paz - nearest hotel: Alojamiento Tokio
miles city - nearest hotel: Historic Olive Hotel
cruz alta - nearest hotel: Hotel e Restaurante Linassi
punta arenas - nearest hotel: Hotel Lacolet
newman - nearest hotel: No hotel found
palmer - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
27,guerrero negro,MX,27.9769,-114.0611,33,Plaza sal paraiso
53,papatowai,NZ,-46.5619,169.4708,49,No hotel found
62,faya-largeau,TD,17.9257,19.1043,28,No hotel found
74,port pirie,AU,-33.1833,138.0167,42,Ellen Hotel
84,aioun,MR,16.6614,-9.6149,12,فندق العيون
95,broken hill,AU,-31.9500,141.4333,38,Ibis Styles
97,laguna,US,38.4210,-121.4238,32,Holiday Inn Express & Suites
102,isla vista,US,34.4133,-119.8610,37,The Club & Guest House
114,karratha,AU,-20.7377,116.8463,39,Karratha International Hotel
119,masterton,NZ,-40.9597,175.6575,48,Copthorne Hotel Solway Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True, 
    tiles="OSM", 
    size="Humidity", 
    color="City", 
    alpha=0.5, 
    hover_cols=["Humidity", "Country", "Hotel Name", "Country"]
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name,Country)